In [23]:
from fss import FSS
from pso import PSO
from initializers import UniformInitializer

In [2]:
fss = FSS(
    num_particles=30,
    num_dims=30,
    initial_weight=1,
    ind_step_range=[0.1, 0.001],
    vol_step_range=[0.1, 0.001],
    step_decay_iterations=1000,
    initializer=UniformInitializer([[-5,5]])
)

In [25]:
from pso.topology import GlobalTopology

In [27]:
pso = PSO(
    num_particles=30,
    num_dims=2,
    initializer=UniformInitializer([[-5,5]]),
    topology=GlobalTopology()
)

In [ ]:
import hp

In [3]:
from test_functions import sphereFunction, rastriginsFunction

In [4]:
fss.initialize()
for i in range(1000):
    fss.minimize(rastriginsFunction)
    print(fss.getBestSolution()[1])

[416.57561136]
[415.15844656]
[411.56082328]
[411.00667063]
[404.70207414]
[403.42362256]
[403.42362256]
[398.28356062]
[396.94873958]
[396.81771914]
[394.9491671]
[394.9491671]
[394.9491671]
[380.8624292]
[377.84351618]
[376.00902563]
[376.00902563]
[374.94476023]
[374.94476023]
[374.94476023]
[374.94476023]
[374.94476023]
[374.94476023]
[374.94476023]
[374.94476023]
[374.94476023]
[374.94476023]
[374.94476023]
[374.71847988]
[374.71847988]
[374.71847988]
[374.71847988]
[374.71847988]
[374.71847988]
[374.71847988]
[374.71847988]
[374.71847988]
[374.71847988]
[371.56641981]
[370.51404879]
[370.51404879]
[363.71547112]
[354.37021279]
[354.37021279]
[349.7869273]
[349.7869273]
[349.7869273]
[349.7869273]
[349.7869273]
[349.7869273]
[349.7869273]
[349.7869273]
[349.7869273]
[349.7869273]
[349.7869273]
[349.7869273]
[349.7869273]
[349.7869273]
[349.7869273]
[349.7869273]
[349.7869273]
[349.7869273]
[349.7869273]
[349.7869273]
[349.7869273]
[349.7869273]
[349.7869273]
[349.7869273]
[349.786

In [5]:
fss.iteration

1000

In [7]:
import numpy as np

In [8]:
import pandas as pd

In [9]:
df = pd.DataFrame([[1,2],[3,4]])

In [22]:
(df[0] > 0) | (df[1] > 0)

0    True
1    True
dtype: bool